<a href="https://colab.research.google.com/github/shivaZeymaran/TRFPre-code-reviewer-recommender/blob/main/TRFPre_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import shutil
import json
import pandas as pd
import math
from datetime import *
from dateutil import parser
import itertools
from collections import defaultdict
import ast
import sklearn
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
dataset = 'openstack'

# Implementation

## Load and Clean data

In [4]:
# Load data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/'+dataset+'_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2011-07-18 15:43:34', 'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612', 'ownerId': '1', 'reviewersId': ['2'], 'filePaths': ['modules/ssh/files/sshd_config', 'manifests/server.pp', 'modules/ssh/manifests/init.pp']}
{'time': '2011-07-18 16:56:22', 'changeId': 'Iea32ad3aedbee996db53655842061d14c9e3d876', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp', 'modules/jenkins_slave/files/known_hosts']}
{'time': '2011-07-18 17:14:05', 'changeId': 'Ib6af08a6af71a6e5853d958a12f63c381eafa039', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 14:37:53', 'changeId': 'Ide7975c522b6b31a23af67b135135c07d75d9bde', 'ownerId': '2', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/files/pubring.gpg', 'modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 15:12:47', 'changeId': 'I28119ac1735127812866a4b0519368a829e2e666', 'ownerId': '2', 'r

In [5]:
# convert times to datetime format
for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612',
 'filePaths': ['modules/ssh/files/sshd_config',
  'manifests/server.pp',
  'modules/ssh/manifests/init.pp'],
 'ownerId': '1',
 'reviewersId': ['2'],
 'time': datetime.datetime(2011, 7, 18, 15, 43, 34)}

In [6]:
len(data)

6545

In [7]:
# Save only PRs with 1 reviwer (to make the results similar to the paper)
final_data = []
for pr in data:
  if len(pr['reviewersId']) > 1:
    revs = pr['reviewersId']
    for r in revs:
      pr['reviewersId'] = [r]
      final_data.append(pr)
  else:
    final_data.append(pr)

len(final_data)

9453

## Approach

In [ ]:
def get_all_reviewers():
  # create a set of all reviewers
  all_reviewers = set()   
  for pr in final_data:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  return all_reviewers

all_reviewers = list(get_all_reviewers())

In [8]:
all_reviewers = ['1030', '2622', '927', '1247', '209', '2035', '452', '100', '543', '191', '2', '321', '704', '1699', '845', '518', '67', '5', '6', '1155', '2284', '706', '357', '1561', '679', '964', '612', '1549', '2472', '616', '233', '8', '1456', '70', '1998', '112', '1216', '4', '979', '335', '2170', '447', '688', '161', '97', '2230', '866', '917', '1192', '1816', '941', '385', '360', '261', '330', '1205', '2325', '1009', '475', '838', '1179', '490', '1689', '2458', '494', '1', '693', '24', '1228', '970', '642', '512', '308', '7', '2243', '236', '995', '34', '1132']

print("All Reviewers in project:", all_reviewers)
print("Number of Reviewers in whole project:", len(all_reviewers))

All Reviewers in project: ['1030', '2622', '927', '1247', '209', '2035', '452', '100', '543', '191', '2', '321', '704', '1699', '845', '518', '67', '5', '6', '1155', '2284', '706', '357', '1561', '679', '964', '612', '1549', '2472', '616', '233', '8', '1456', '70', '1998', '112', '1216', '4', '979', '335', '2170', '447', '688', '161', '97', '2230', '866', '917', '1192', '1816', '941', '385', '360', '261', '330', '1205', '2325', '1009', '475', '838', '1179', '490', '1689', '2458', '494', '1', '693', '24', '1228', '970', '642', '512', '308', '7', '2243', '236', '995', '34', '1132']
Number of Reviewers in whole project: 79


In [9]:
# problem hyper-parameters
lambda_val = 30  # can be changed
gama = 0.7

### Time-decay Relationship

In [ ]:
def find_PRlist(pull_reqs, reviewerId, cur_time):

  # lambda days before the time of our test PR
  lambda_ago = cur_time - timedelta(days=lambda_val)

  # list of PRs in "pull_reqs" that are between time of lambda days ago up to the time of test PR
  PS = [pr for pr in pull_reqs if ((lambda_ago < pr['time'] < cur_time) and (reviewerId in pr['reviewersId']))]
  # print("size of final list:", len(PS))

  # PS will be the list of PRs which submitted by the owner of test PR and reviewed by any reviewer in lambda days ago
  return PS 

In [ ]:
def cal_decayRelation(pull_reqs, cur_time):
  return sum([(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

### Time-decay File Similarity

In [ ]:
def pathDistance(f1, f2):
  
  commen_prefix = os.path.commonprefix([f1, f2]).split("/")
  commen_len = len(commen_prefix)
  if '' in commen_prefix:
   commen_len -= 1
  # print(commen_prefix, commen_len)

  max_len = max(len(f1.split("/")), len(f2.split("/")))
  # print(max_len)

  return commen_len / max_len

In [ ]:
def prDistance(pr1, pr2):
  return sum([pathDistance(f1, f2) for f1 in pr1['filePaths'] for f2 in pr2['filePaths']]) / (len(pr1['filePaths'])*len(pr2['filePaths']))

In [ ]:
def cal_decayFileSimilarity(pull_reqs, cur_time, test_pr):
  return sum([prDistance(test_pr, pr)*(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

### Feature Extraction
Construct a Pull Request Vector for each PR

In [ ]:
# For all PRs we should extract PRvector and save this and reviewer ID into a dataframe for future use

vectors = []
labels = []

for t in range(0, len(final_data)):

  prev_pulls = final_data[0:t]
  cur_pull = final_data[t]

  # list of PRs that submitted by the owner of this PR in trainset
  OS = [pr for pr in prev_pulls if (pr['ownerId'] == cur_pull['ownerId'])]

  cur_time = cur_pull['time']

  # Calculating time-decaying Relationship
  PRvector = [cal_decayRelation(find_PRlist(OS, rev, cur_time), cur_time) for rev in all_reviewers]

  # Calculating time-decaying File Similarity
  PRvector.extend([cal_decayFileSimilarity(find_PRlist(prev_pulls, rev, cur_time), cur_time, cur_pull) for rev in all_reviewers])

  if t % 200 == 0:
    print("PR Vector #", t, ": ", PRvector)

  # Save to dataframe
  vectors.append(PRvector)
  labels.append(cur_pull['reviewersId'][0])


print("PR vector size:", len(all_reviewers), "is equal to: 2 *", len(all_reviewers), "=", 2*len(all_reviewers))
print(len(vectors), len(labels))

PR Vector # 0 :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# Create a dataframe of vectors and labels(reviewerIDs)
df = pd.DataFrame()
df['PRvector'] = vectors
df['reviewerID'] = labels
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",216296
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951


In [ ]:
# Save dataframe as csv file
df.to_csv('dataframe.csv')

In [ ]:
# Save dataframe into Drive
src = "./dataframe.csv"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/"+dataset+"_FeatureVectors.csv"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/angular.js_FeatureVectors.csv'

### Experiment & Result

In [10]:
def pred_classes(index):
  all_reviewers = set()   
  for pr in final_data[0:index]:
    for r in pr['reviewersId']:
      all_reviewers.add(int(r))

  return sorted(all_reviewers)

In [11]:
# Function to write predicted top-k reviewers for each test into a JSON file
def save_predictions(pred_list):
  with open('output.json', 'a') as outfile:
    json.dump(pred_list, outfile, indent=4)

In [12]:
def create_pred_obj(group_num, test_num, pred_revs, actual_rev):
  pred_obj = {}
  pred_obj['changeId'] = str(group_num) + "g" + str(test_num) 
  pred_obj['actual'] = [actual_rev]
  pred_obj['recommended'] = pred_revs
  return pred_obj

## Main

In [13]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project/openstack_FeatureVectors.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [14]:
# PRs created in the same month should be in the same group
groups = []

start_year = final_data[0]['time'].year
start_month = final_data[0]['time'].month
group = []

for i in range(0, len(final_data)):
  if (final_data[i]['time'].year == start_year) and (final_data[i]['time'].month == start_month):
    group.append(df.loc[i, :])
    if(i == len(final_data)-1): # for the last PR
      groups.append(group)
  else:
    groups.append(group)
    group = []
    start_year = final_data[i]['time'].year
    start_month = final_data[i]['time'].month
    group.append(df.loc[i, :])

len(groups)

11

In [15]:
rounds_num = len(groups)-1

RFmodel = RandomForestClassifier(n_estimators=12, max_depth=None, random_state=7)
NBmodel = GaussianNB()
# SVMmodel = svm.SVC(probability=True)

train = groups[0]
# setting X, y to use for model construction
X = [ast.literal_eval(train[j]["PRvector"]) for j in range(0, len(train))]
y = [train[j]["reviewerID"] for j in range(0, len(train))]

for i in range(0, rounds_num):

  predictions_list = []  # final predicted reviewers for each test PR

  print("********** Test Group number", i+1, "**********")

  test = groups[i+1]
  X_test = [ast.literal_eval(test[j]["PRvector"]) for j in range(0, len(test))]
  y_test = [test[j]["reviewerID"] for j in range(0, len(test))]

  # use classifiers to find probabilities for each test
  for t in range(0, len(X_test)):

    # Random Forest classifier
    RFmodel.fit(X, y)
    RF_prob = RFmodel.predict_proba(np.reshape(X_test[t], (1, -1)))

    # Naive Bayes classifier
    NBmodel.fit(X, y)
    NB_prob = NBmodel.predict_proba(np.reshape(X_test[t], (1, -1)))

    # SVM classifier
    # SVMmodel.fit(X, y)
    # SVM_prob = SVMmodel.predict_proba(np.reshape(X_test[t], (1, -1)))
    
    # Composer
    comp_prob = (RF_prob * gama) + (NB_prob * (1-gama))
    comp_prob = [round(c, 2) for c in comp_prob[0]]

    # dictionary of each label and its probability
    results = dict(zip(pred_classes(len(X)), comp_prob))  
    # sort predicted reviewers by probability values
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    # extract maximum 10 first predictions
    pred_revs = [str(tup[0]) for tup in sorted_results[0:10] if (tup[1]!=0)]

    if t % 10 == 0:
      print("Test number", t)
      print("Predicted reviewers:", pred_revs)
      print("Actual reviewer:", str(y_test[t]))
      print("-----------------")

    # make object of real and predicted reviewers
    predictions_list.append(create_pred_obj(i+1, t, pred_revs, str(y_test[t])))

    # append the test PR to trainset
    X.append(X_test[t])
    y.append(y_test[t])


  # save objects into a file
  save_predictions(predictions_list)


********** Test Group number 1 **********
Test number 0
Predicted reviewers: ['5', '2', '1', '7']
Actual reviewer: 5
-----------------
Test number 10
Predicted reviewers: ['2', '5', '1', '7', '4']
Actual reviewer: 2
-----------------
Test number 20
Predicted reviewers: ['5', '4', '7', '1', '2']
Actual reviewer: 1
-----------------
Test number 30
Predicted reviewers: ['5', '2', '7', '1', '4', '1132']
Actual reviewer: 7
-----------------
Test number 40
Predicted reviewers: ['5', '4', '1', '7', '2', '475', '1132']
Actual reviewer: 7
-----------------
Test number 50
Predicted reviewers: ['5', '1132', '1', '7', '2', '475', '4']
Actual reviewer: 1
-----------------
Test number 60
Predicted reviewers: ['4', '1', '5', '7', '475', '706', '1132', '2']
Actual reviewer: 4
-----------------
Test number 70
Predicted reviewers: ['5', '1', '4', '7', '2', '475', '706', '1132']
Actual reviewer: 5
-----------------
Test number 80
Predicted reviewers: ['4', '5', '7', '1', '1132', '2', '475', '706']
Actual

KeyboardInterrupt: ignored